In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from lerobot.src.classes.corexy_stylus_teleop import (
    CoreXYStylusTeleopConfig, CoreXYStylusTeleop
)

teleop = CoreXYStylusTeleop(CoreXYStylusTeleopConfig())
teleop.connect()

try:
    while teleop._running:
        action = teleop.get_action()
        if action["tap"]: # ignore idle frames
            pass
finally:
    teleop.disconnect()

In [ ]:
from lerobot.src.classes.corexy_stylus_robot import (
    CoreXYStylusRobotConfig, CoreXYStylusRobot
)

robot  = CoreXYStylusRobot(CoreXYStylusRobotConfig(port="COM5"))
robot.connect(calibrate=True)

try:
    while True:
        robot.send_action({"x": 0.45, "y": 0.45, "tap": 1})
        robot.send_action({"x": 0.55, "y": 0.55, "tap": 1})
finally:
    teleop.disconnect()
    robot.disconnect()

In [ ]:
from lerobot.src.classes.corexy_stylus_teleop import (
    CoreXYStylusTeleopConfig, CoreXYStylusTeleop
)
from lerobot.src.classes.corexy_stylus_robot import (
    CoreXYStylusRobotConfig, CoreXYStylusRobot
)

teleop = CoreXYStylusTeleop(CoreXYStylusTeleopConfig())
robot  = CoreXYStylusRobot(CoreXYStylusRobotConfig(port="COM5"))

teleop.connect()
robot.connect(calibrate=True)

try:
    while teleop._running:
        action = teleop.get_action()
        if action["tap"]:          # ignore idle frames
            #robot.send_action(action)
            pass
finally:
    teleop.disconnect()
    robot.disconnect()


In [ ]:
import cv2
import numpy as np
print("OpenCV:", cv2.__version__, "| NumPy:", np.__version__)
cv2.namedWindow("dummy")  # should succeed
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,  1920)   # request 1080p; harmless if ignored
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

ret, frame = cap.read()
if not ret:
    raise RuntimeError("No video signal from capture card")

# Any pixel with luma > threshold is “active”; tweak thr if your bars are grey
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
thr  = 20 # 0–255; raise if bars look grey
ys, xs = np.where(gray > thr)

y0, y1 = ys.min(), ys.max()
x0, x1 = xs.min(), xs.max()
roi    = np.s_[y0:y1+1, x0:x1+1] # slice we’ll apply each frame
Hc, Wc = y1 - y0 + 1, x1 - x0 + 1
print(f"Active picture: {Wc}×{Hc}  (cropped {frame.shape[1]}×{frame.shape[0]})")

# 4:3 sanity check (optional)
print(f"Aspect ≈ {Wc/Hc:.2f}  (expected 1.33 for 4∶3)")

cv2.namedWindow("iPad – clean 4:3", cv2.WINDOW_NORMAL)
cv2.resizeWindow("iPad – clean 4:3", Wc, Hc)

while True:
    ret, frame = cap.read()
    if not ret: break

    ipad = frame[roi] # fast NumPy view – no copy
    cv2.imshow("iPad – clean 4:3", ipad)

    k = cv2.waitKey(1) & 0xFF
    if k == 27: break # Esc


cap.release(); cv2.destroyAllWindows()

In [ ]:
import cv2, threading, time

fps = 30

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,  1920)   # request 1080p; harmless if ignored
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

def _window_loop():
    cv2.setMouseCallback("Test", _on_mouse)
    
    while True:
        ret, frame = cap.read()
        cv2.imshow("Test", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit window
            break
        time.sleep(1 / fps)

def _on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Mouse clicked at {x}, {y}")

ret, frame = cap.read()
cv2.imshow("Test", frame)

threading.Thread(target=_window_loop, daemon=True).start()

while True:
    time.sleep(0.1)


In [2]:
import cv2, threading, time

fps = 30

cap1 = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap1.set(cv2.CAP_PROP_FRAME_WIDTH,  1920)   # request 1080p; harmless if ignored
cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

cap2 = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap2.set(cv2.CAP_PROP_FRAME_WIDTH,  1920)   # request 1080p; harmless if ignored
cap2.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

def _window_loop1():
    cv2.setMouseCallback("Test", _on_mouse1)
    
    while True:
        ret, frame = cap1.read()
        cv2.imshow("Test", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit window
            break
        time.sleep(1 / fps)

def _window_loop2():
    cv2.setMouseCallback("Test 2", _on_mouse2)
    
    while True:
        ret, frame = cap2.read()
        cv2.imshow("Test 2", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit window
            break
        time.sleep(1 / fps)

def _on_mouse1(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Mouse clicked at {x}, {y} in Test window")

def _on_mouse2(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Mouse clicked at {x}, {y} in Test 2 window")

ret, frame = cap1.read()
cv2.imshow("Test", frame)

ret, frame = cap2.read()
cv2.imshow("Test 2", frame)

threading.Thread(target=_window_loop1, daemon=True).start()
#threading.Thread(target=_window_loop2, daemon=True).start()

while True:
    time.sleep(0.1)

: 